In [1]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/01_RBHs")

library(data.table)

myo_meta <- read.csv("/Users/rebecca/sudmant/analyses/myotis/data/myotis_meta.csv")
ortho_table <- fread("/Users/rebecca/sudmant/analyses/myotis/data/ortholog_mapping.csv", data.table = FALSE)
yum_col <- grep("^yum", colnames(ortho_table))
ortho_table[,yum_col] <- gsub("SCAF", "SUPER", ortho_table[,yum_col]) 
colnames(ortho_table)[1:3] <- c("ENSEMBL", "SYMBOL", "Myotis_Alias")

In [2]:
## Intersect RBH peaks with orthologous genes

for (i in seq_along(1:nrow(myo_meta))) {

  spec_id <- tolower(gsub("Myotis-", "", myo_meta$Field_Name[i]))
  peak_files <- list.files(path = "results/data", 
                           pattern = paste0("^", myo_meta$Abbr[i], ".*_genes.*_5000.*.csv"),
                           full.names = TRUE)
  
  for(j in seq_along(peak_files)) {

    ## Parse gene column:
    
    peaks <- fread(peak_files[j], data.table = FALSE)
    
    if (grepl("exon", peak_files[j])) {
      
      peaks$Gene <- sapply(strsplit(peaks$Gene, ";"), "[", 2)
      peaks$Gene <- gsub("model", "TU", peaks$Gene)
      
    } else {
      peaks$Gene <- sapply(strsplit(peaks$Gene, ";", fixed = TRUE), "[", 1)
    }                                             
    peaks$Gene <- gsub(".*=", paste0(spec_id, "_"), peaks$Gene)    
                                                       
    ## Intersect best hit genes with orthologous genes:
    spec_col <- grep(paste0(spec_id, "_ID"), colnames(ortho_table))
    spec_ortho_table <- ortho_table[,c(1:3, spec_col),]
    colnames(spec_ortho_table)[4] <- "Gene"
    spec_ortho_table <- spec_ortho_table[spec_ortho_table$Gene %in% peaks$Gene,]
    peak_orthos <- merge(peaks, spec_ortho_table, by = "Gene", all.x = TRUE, sort = FALSE)
    
    fwrite(peak_orthos, file = gsub(".csv", "_orthologs.csv", peak_files[j]))

  }

}                                